# Author

- Name: **Chaipat Suwannapoom**
- Linkedin: https://www.linkedin.com/in/bchaipats/
- GitHub: https://github.com/bchaipats
- Medium: https://medium.com/@bchaipats
- Twitter: https://twitter.com/bchaipats

# Overview

In this notebook, we'll be creating a personal assistant powered by open-source LLMs, free of charge.

# Set up

In [ ]:
# Install dependencies
!pip install langchain gpt4all llama-index sentence-transformers

# Run LLM on a personal laptop

If you worry about your data privacy (and your money), you might prefer running open-source LLMs privately on your local laptop instead of making API calls to external services like OpenAI.

GPT4All helps us achieve this. We can browse and download open-source LLMs from the [GPT4All homepage](https://gpt4all.io/index.html) into our laptop. Say, we pick Mistral-7B since it's so cool and requires only 8 GB of RAM which is suitable for most consumer-grade CPUs.

In [ ]:
# Download an open-source LLM
!wget https://gpt4all.io/models/gguf/mistral-7b-openorca.Q4_0.gguf

In [1]:
from langchain.llms import GPT4All

# Load the model
llm = GPT4All(model='./mistral-7b-openorca.Q4_0.gguf')

# Define a sentence to complete
sentence = "The capital of Thailand is "

# Complete the sentence
response = llm.generate([sentence])
print(response.generations[0][0].text)

 Bangkok, which is also known as the City of Angels. It’s a bustling city with over 10 million people and it can be overwhelming at first glance. But don’t worry! I have put together this list of things to do in Bangkok for you so that your trip will be smooth sailing.



# Personal data

We'll be using a simple text file as our data source for simplicity. Later, we can just copy and paste texts from personal calendar, news, articles, books, etc. For example, here is a mock calendar for testing:

In [2]:
%%writefile ./data.txt
# data.txt

19 Dec - Go to the movie
20 Dec - Have a dinner with family
21 Dec - Go to the birthday party
22 Dec - Go to the dentist
23 Dec - Finish writing a draft for blog post

Overwriting ./data.txt


# Create necessary components
- **Embedding model** used to generate embedding vector representing the original text.
- **Prompt helper** used to handle LLM context window and token limitations.
- **Sentence splitter** used to split data into multiple chunks.

In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index import PromptHelper
from llama_index.node_parser import SentenceSplitter

# An embedding model used to structure text into representations
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

# PromptHelper can help deal with LLM context window and token limitations
prompt_helper = PromptHelper(context_window=2048)

# SentenceSplitter used to split our data into multiple chunks
# Only a number of relevant chunks will be retrieved and fed into LLMs
node_parser = SentenceSplitter(chunk_size=300, chunk_overlap=20)

/Users/bcp/medium/medium-notebooks/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create a container service
This service packs the large language model, the embedding model, the prompt helper and the node parser into a single handy container.

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    prompt_helper=prompt_helper,
    node_parser=node_parser)

# Build an index and a query engine

In [5]:
from llama_index import SimpleDirectoryReader
from llama_index import VectorStoreIndex

# Load data.txt into a document
document = SimpleDirectoryReader(input_files=['./data.txt']).load_data()

# Process data (chunking, embedding, indexing) and store them
index = VectorStoreIndex.from_documents(
    document, service_context=service_context)

# Build a query engine from the index
query_engine = index.as_query_engine()

In [6]:
response = query_engine.query('Give me my calendar.')
print(response)


Your calendar for December is as follows:
19 Dec - Go to the movie
20 Dec - Have a dinner with family
21 Dec - Go to the birthday party
22 Dec - Go to the dentist
23 Dec - Finish writing a draft for blog post


In [7]:
response = query_engine.query("What's the plan for 22 Dec?")
print(response)

22 Dec is planned as "Go to the dentist".


In [8]:
response = query_engine.query('When should I finish a writing draft?')
print(response)

23 Dec


# Summarize news

For text summarization, it is better to create a new index that is suitable for the task. This is called a **SummaryIndex** which will synthesize an answer from all chunks of data compared to top-k chunks as in **VectorStoreIndex**.

Credit: the following is the text from this [news](https://www.newsinlevels.com/products/grenade-in-the-face-level-3/).

In [9]:
%%writefile ./data.txt
# data.txt

Grenade in the face – level 3

A soldier is lucky to be alive after a 40-millimetre grenade accidentally fired from a grenade launcher and embedded itself in his face.

The terrible accident happened in Colombia and surgeons had to be especially careful with the operation.

The soldier could not be moved to hospital by helicopter due to the risk of the explosive detonating, so an ambulance transferred him in an eight-hour ride to Bogota.

The team of doctors had to improvise and operate outside the hospital to minimise impact on the building in case the grenade exploded. Luckily, the operation was a success and the soldier could reintegrate into his military unit and his family to make a full recovery.

Overwriting ./data.txt


In [10]:
from llama_index import SummaryIndex

# Load data.txt into a document
document = SimpleDirectoryReader(input_files=['./data.txt']).load_data()

# Process data (chunking, embedding, indexing) and store them
summary_index = SummaryIndex.from_documents(
    document, service_context=service_context)

# Build a summary engine from the index
summary_engine = summary_index.as_query_engine(response_mode="tree_summarize")

In [11]:
response = summary_engine.query("What is a summary of this collection of text?")
print(response)


A soldier in Colombia survived an accident where a grenade fired from a launcher embedded itself into his face. Due to the risk of explosion, he was transported by ambulance instead of helicopter and underwent surgery outside the hospital. The operation was successful, allowing him to recover and reintegrate with his military unit and family.
